In [1]:
import pandas as pd
import html5lib
import requests
import string
from bs4 import BeautifulSoup
import sqlite3 as lite
from pandas.io import sql

import numpy as np
from pandas import DataFrame, Series, read_csv
import matplotlib.pyplot as plt
import matplotlib 
import re
import itertools

# Enable inline plotting
%matplotlib inline

# Extend the Limit of rows
pd.options.display.max_rows = 150

In [2]:
url= 'http://www.acaloriecounter.com/fast-food.php'
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html5lib')

In [3]:
# returns titles in a list named TABLE_TITLE.
TABLE_TITLE = []
for item in soup.find_all('h2'):
    TABLE_TITLE.append(item.text.rstrip('\n'))

# |REMOVING LAST TWO ROWS FROM THE TABLE TITLE LIST. #RETURNS COMPELTE LIST OF TITLE 
# |TABLES ON SITE.
TABLE_TITLE = TABLE_TITLE[:10]

# printing the results of the categories.
counter = 0
for title in TABLE_TITLE:
    counter += 1
    print ('Table #:| Category: ')
    print (counter,' :     ', title)
    print ('-'*70)
    print ('\n')

Table #:| Category: 
1  :      French Fries (Large) Compared
----------------------------------------------------------------------


Table #:| Category: 
2  :      Hamburgers Compared
----------------------------------------------------------------------


Table #:| Category: 
3  :      Sandwiches/Hamburgers Compared
----------------------------------------------------------------------


Table #:| Category: 
4  :      Chicken (Individual Pieces) Compared
----------------------------------------------------------------------


Table #:| Category: 
5  :      Chicken Sandwiches Compared
----------------------------------------------------------------------


Table #:| Category: 
6  :      Onion Rings Compared
----------------------------------------------------------------------


Table #:| Category: 
7  :      Breakfast Sandwiches/Biscuits/Croissants/Wraps Compared
----------------------------------------------------------------------


Table #:| Category: 
8  :      Mozzarella Sticks 

In [4]:
TABLE_CATEGORIES = ['Restaurants',
             'Type',
             'Serving_Size',
             'Calories',
             'Total_Fat',
             'Saturated_Fat',
             'Trans_Fat',
             'Carbs',
             'Sodium'
            ]

In [5]:
#Soup object returns all rows from all tables and saves the to a list.
TABLES_DATA = []
for tables in soup.find_all("table"):
    table = []
    for rows in tables.find('tbody'):
        for items in rows:
            for text in items:
                if text == ' ':
                    pass
                elif text == '\n':
                    pass
                elif text == '\t':
                    pass
                else:
                    table.append(text.replace('*', '').replace('\n', ''))
    TABLES_DATA.append(table)

    counter = 0

#displays the TABLE TITLE and corresponding first ROWS of the Table.
for table in TABLES_DATA:
    print ('\n')
    print ('Table : ', TABLE_TITLE[counter])
    print ('-'*70)
    counter += 1
    for row in table[:9]:
        print (row)
    



Table :  French Fries (Large) Compared
----------------------------------------------------------------------
McDonald's
Regular
170
570
30
6
8
70
330


Table :  Hamburgers Compared
----------------------------------------------------------------------
McDonald's
Regular
100
250
9
3.5
0.5
31
520


Table :  Sandwiches/Hamburgers Compared
----------------------------------------------------------------------
McDonald's
Quarter Pounder
169
410
19
7
1
37
730


Table :  Chicken (Individual Pieces) Compared
----------------------------------------------------------------------
McDonald's
Chicken McNuggets (6pc)
96
250
15
3
1.5
15
670


Table :  Chicken Sandwiches Compared
----------------------------------------------------------------------
McDonald's
McChicken
147
360
16
3.5
1
40
790


Table :  Onion Rings Compared
----------------------------------------------------------------------
Burger King (King Size)
150
500
25
5
4.5
62
720
A&W


Table :  Breakfast Sandwiches/Biscuits/Croissants/

In [6]:
Table_Names = ['Fries',
               'Hamburgers',
               'Sandwiches_Hamburger',
               'Chicken_Pieces',
               'Chicken_Sandwiches',
               'Onion_Rings',
               'Bkfst_Sandwiches',
               'Mozzarella_Sticks',
               'BreadSticks_CheesyBread',
               'Pizza_Large14'
              ]


In [7]:
def parse_table(table):
    results = []
    counter = 0
    table_row = []
    for row in table:
        try:
            table_row.append(float(row))
        except Exception as e:
            table_row.append(row)
        counter += 1
        if counter % 9 == 0:
            results.append(table_row)
            table_row = []
    return results

In [8]:
def easy_dataframe(table_list, table_catg):
    df_object = pd.DataFrame(table_list, columns = table_catg)
    return df_object.replace('Unknown', 0)

In [9]:
counter = 0
for item in Table_Names:
    exec('{} = easy_dataframe({},{})'.format(item.strip(),
                                             parse_table(TABLES_DATA[counter]),
                                             TABLE_CATEGORIES))
    counter += 1

In [10]:
#During this process, there is a discrepancy that must be handled so that all the tables are formated correctly
# Onion Rings does not have a Type column, and there it has not formated properly.
# Some additional parsing and custom function to solve this for ONION Rings at this point.
def parse_onions(table):
    results = []
    counter = 0
    table_row = []
    for row in table:
        table_row.append(row)
        counter += 1
        if counter % 8 == 0:
            results.append(table_row)
            table_row = []
    return results

ONION_CATEGORIES = ['Restaurants',
             'Serving_Size',
             'Calories',
             'Total_Fat',
             'Saturated_Fat',
             'Trans_Fat',
             'Carbs',
             'Sodium'
            ]
Onion_Rings = easy_dataframe(parse_onions(TABLES_DATA[5]), ONION_CATEGORIES)

In [11]:
Fries

,Restaurants,Type,Serving_Size,Calories,Total_Fat,Saturated_Fat,Trans_Fat,Carbs,Sodium
0,McDonald's,Regular,170,570,30,6.0,8.0,70,330
1,Burger King,Regular,160,500,28,6.0,6.0,57,820
2,Burger King,Salt Not Added,160,500,28,6.0,6.0,57,530
3,Wendy's,Regular,0,540,26,4.0,1.0,69,550
4,Arby's,Homestyle,213,566,37,7.0,1.0,82,1029
5,Arby's,Curly,198,631,37,7.0,1.0,73,1476
6,Hardee's,Regular,193,610,28,6.0,0.0,78,370
7,Hardee's,Crispy Curls,153,480,23,6.0,0.0,60,1190
8,A&W,Regular,156,430,18,4.5,5.5,61,640
9,A&W,Chili,170,370,16,4.5,6.0,49,780


In [12]:
Fries.dtypes

Restaurants       object
Type              object
Serving_Size     float64
Calories         float64
Total_Fat        float64
Saturated_Fat    float64
Trans_Fat        float64
Carbs            float64
Sodium           float64
dtype: object

In [13]:
CarbsCal = Fries['Carbs'] * 4
FatCal = Fries['Total_Fat'] * 9

#figuring out protien calories
TotalKnownCal = CarbsCal + FatCal
ProteinCal = Fries['Calories'] - TotalKnownCal

#converting protien to gram
Protein = ProteinCal / 4

#adding ratios
CarbsRatio = CarbsCal / Fries['Calories']
FatRatio = FatCal / Fries['Calories'] 
ProteinRatio = ProteinCal / Fries['Calories'] 


In [31]:
Fries.insert(4,'Protein', Protein)
Fries["Carb_Ratio"] = CarbsRatio * 100
Fries["Fat_Ratio"] = FatRatio * 100
Fries["Protein_Ratio"] = ProteinRatio * 100
Fries

,Restaurants,Type,Serving_Size,Calories,Protein,Total_Fat,Saturated_Fat,Trans_Fat,Carbs,Sodium,Carb_Ratio,Fat_Ratio,Protein_Ratio
0,McDonald's,Regular,170,570,5.00,30,6.0,8.0,70,330,49.122807,47.368421,3.508772
1,Burger King,Regular,160,500,5.00,28,6.0,6.0,57,820,45.600000,50.400000,4.000000
2,Burger King,Salt Not Added,160,500,5.00,28,6.0,6.0,57,530,45.600000,50.400000,4.000000
3,Wendy's,Regular,0,540,7.50,26,4.0,1.0,69,550,51.111111,43.333333,5.555556
4,Arby's,Homestyle,213,566,-23.75,37,7.0,1.0,82,1029,57.950530,58.833922,-16.784452
5,Arby's,Curly,198,631,1.50,37,7.0,1.0,73,1476,46.275753,52.773376,0.950872
6,Hardee's,Regular,193,610,11.50,28,6.0,0.0,78,370,51.147541,41.311475,7.540984
7,Hardee's,Crispy Curls,153,480,8.25,23,6.0,0.0,60,1190,50.000000,43.125000,6.875000
8,A&W,Regular,156,430,6.00,18,4.5,5.5,61,640,56.744186,37.674419,5.581395
9,A&W,Chili,170,370,7.50,16,4.5,6.0,49,780,52.972973,38.918919,8.108108


In [15]:
Hamburgers

,Restaurants,Type,Serving_Size,Calories,Total_Fat,Saturated_Fat,Trans_Fat,Carbs,Sodium
0,McDonald's,Regular,100,250,9,3.5,0.5,31,520
1,Burger King,Regular,121,290,12,4.5,0.0,30,560
2,Wendy's,Jr. Hamburger,0,230,8,3.0,0.0,26,500
3,Hardee's,Regular,118,310,12,4.0,0.0,36,560
4,Carl's Jr,Big Hamburger,209,470,17,6.0,0.0,54,1060
5,Sonic,Jr. Burger,117,310,15,5.0,0.5,30,610
6,White Castle,Regular,58,140,7,2.5,0.5,14,210
7,Dairy Queen,Homestyle Burger,140,350,14,7.0,0.5,33,400
8,Jack in the Box,Regular,118,310,14,6.0,1.0,30,600
9,In-N-Out Burger,Regular,243,390,19,5.0,0.0,39,650


In [16]:
Hamburgers.dtypes

Restaurants       object
Type              object
Serving_Size     float64
Calories         float64
Total_Fat        float64
Saturated_Fat    float64
Trans_Fat        float64
Carbs            float64
Sodium           float64
dtype: object

In [17]:
Sandwiches_Hamburger

,Restaurants,Type,Serving_Size,Calories,Total_Fat,Saturated_Fat,Trans_Fat,Carbs,Sodium
0,McDonald's,Quarter Pounder,169,410,19,7,1.0,37,730
1,McDonald's,Quarter Pounder (cheese),198,510,26,12,1.5,40,1190
2,McDonald's,Double Quarter Pounder (cheese),279,740,42,19,2.5,40,1380
3,McDonald's,Big Mac,214,540,29,10,1.5,43,1040
4,McDonald's,Double Cheeseburger,165,440,23,11,1.5,34,1150
5,McDonald's,Big N' Tasty,206,460,24,8,1.5,37,720
6,McDonald's,Big N' Tasty (cheese),220,510,28,11,1.5,38,960
7,Burger King,Whopper,290,670,39,11,1.5,51,1020
8,Burger King,Whopper (cheese),315,760,47,16,1.5,52,1450
9,Burger King,Double Whopper,373,900,57,19,2.0,51,1090


In [18]:
Chicken_Pieces

,Restaurants,Type,Serving_Size,Calories,Total_Fat,Saturated_Fat,Trans_Fat,Carbs,Sodium
0,McDonald's,Chicken McNuggets (6pc),96,250,15,3.0,1.5,15,670
1,McDonald's,Chicken Selects Premium Strips (5pc),221,630,33,6.0,4.5,46,1550
2,Burger King,Chicken Tenders (6pc),92,250,15,3.5,2.5,16,720
3,Burger King,Chicken Fries (6pc),85,260,15,3.5,3.0,18,650
4,Wendy's,Chicken Nuggets (5pc),0,230,15,3.0,0.0,12,520
5,Arby's,Chicken Tenders (5pc),218,630,31,5.0,0.0,47,1977
6,Arby's,Popcorn Chicken (large),184,531,26,6.0,1.0,39,1666
7,Hardee's,Chicken Strips (5pc),241,630,34,6.0,0.0,45,2260
8,Dairy Queen,Chicken Strip Basket (6pc),531,1270,67,11.0,12.0,121,2910
9,KFC,Popcorn Chicken (large),160,550,35,6.0,0.0,30,1600


In [19]:
Chicken_Sandwiches

,Restaurants,Type,Serving_Size,Calories,Total_Fat,Saturated_Fat,Trans_Fat,Carbs,Sodium
0,McDonald's,McChicken,147,360,16.0,3.5,1.0,40,790
1,McDonald's,Premium Grilled Chicken Classic,226,420,10.0,2.0,0.0,51,1190
2,McDonald's,Premium Crispy Chicken Classic,229,500,17.0,3.5,1.5,61,1330
3,McDonald's,Premium Grilled Chicken Club,260,570,21.0,7.0,0.0,52,1720
4,McDonald's,Premium Crispy Chicken Club,263,660,28.0,8.0,1.5,63,1860
5,Burger King,TENDERGRILL (with mayo),258,510,19.0,3.5,0.5,49,1180
6,Burger King,TENDERCRISP,284,790,44.0,8.0,4.0,68,1640
7,Burger King,Original,219,660,40.0,8.0,2.5,52,1440
8,Burger King,Chick'n Crisp (with mayo),144,480,31.0,5.0,2.0,36,870
9,Wendy's,Ultimate Chicken Grill,0,320,7.0,1.5,0.0,36,950


In [20]:
Onion_Rings

,Restaurants,Serving_Size,Calories,Total_Fat,Saturated_Fat,Trans_Fat,Carbs,Sodium
0,Burger King (King Size),150,500,25,5,4.5,62,720
1,A&W,113,350,16,3.5,4.5,45,710
2,Jack In The Box,119,500,30,6,10,51,420
3,White Castle (Sack),178,410,20,5,7,53,400
4,"White Castle (Homestyle, Sack)",178,790,44,8,30,91,860
5,Sonic (Large),227,640,31,5,0.5,80,300
6,Dairy Queen (Large),142,590,37,7,9,56,930
7,Carl's Jr.,128,430,21,4,0,53,550


In [21]:
Bkfst_Sandwiches

,Restaurants,Type,Serving_Size,Calories,Total_Fat,Saturated_Fat,Trans_Fat,Carbs,Sodium
0,McDonald's,Egg McMuffin,139,300,12,5.0,0.0,30,820
1,McDonald's,Sausage McMuffin,114,370,22,8.0,0.0,29,850
2,McDonald's,Sausage McMuffin (egg),164,450,27,10.0,0.0,30,920
3,McDonald's,"Bacon, Egg & Cheese Biscuit (regular)",144,450,25,11.0,0.0,36,1360
4,McDonald's,"Sausage Biscuit (egg, regular)",159,500,32,12.0,0.0,35,1130
5,McDonald's,Sausage Biscuit (regular),113,410,27,10.0,0.0,33,1040
6,McDonald's,Biscuit (regular),72,250,11,5.0,0.0,32,700
7,McDonald's,"Bacon, Egg & Cheese McGriddles",173,460,21,9.0,0.0,48,1360
8,McDonald's,"Sausage, Egg & Cheese McGriddles",202,560,32,12.0,0.0,48,1360
9,McDonald's,Sausage McGriddles,141,420,22,8.0,0.0,44,1030


In [22]:
Mozzarella_Sticks

,Restaurants,Type,Serving_Size,Calories,Total_Fat,Saturated_Fat,Trans_Fat,Carbs,Sodium
0,Arby's,4 pc (Regular),137.0,426,28,13,1.0,38,1370
1,Arby's,8 pc (Large),273.0,849,56,26,2.0,75,2730
2,Jack In The Box,3 pc,71.0,240,12,5,2.0,21,420
3,Jack In The Box,6 pc,138.0,483,27,11,4.0,39,1018
4,White Castle,3 pc,79.2,250,14,6,2.0,22,750
5,White Castle,5 pc,132.0,420,23,10,3.0,37,1240
6,White Castle,10 pc,264.0,820,46,20,6.0,73,2490
7,Sonic,0,140.0,440,22,9,0.5,40,1050


In [23]:
BreadSticks_CheesyBread

,Restaurants,Type,Serving_Size,Calories,Total_Fat,Saturated_Fat,Trans_Fat,Carbs,Sodium
0,Pizza Hut,Breadsticks,1 pc,150,6,1.0,0,20,230
1,Pizza Hut,Cheese Breadsticks,1 pc,200,10,3.0,0,21,370
2,Domino's Pizza,Breadsticks,1 pc,130,7,1.5,0,14,90
3,Domino's Pizza,Cheesy Bread,1 pc,140,7,2.3,0,14,140
4,Little Caesars,Crazy Bread,1 pc,100,3,0.5,0,15,150
5,Little Caesars,Italian Cheese Bread,1 pc,130,7,2.5,0,13,230
6,Little Caesars,Pepperoni Cheese Bread,1 pc,150,8,3.0,0,13,280
7,Papa John's,Breadsticks,1 pc,140,2,0.0,0,26,260
8,Papa John's,Cheesesticks,2 pc,370,16,4.5,0,42,830


In [24]:
Pizza_Large14

,Restaurants,Type,Serving_Size,Calories,Total_Fat,Saturated_Fat,Trans_Fat,Carbs,Sodium
0,Pizza Hut,"14"" Large Pan Pizza",146,390,19,7.0,0,38,800
1,Domino's Pizza,"14"" Classic Hand-Tossed",121,290,9,3.5,0,42,470
2,Papa John's,"14"" Original Crust Pizza",132,300,11,3.5,0,39,750
3,Little Caesars,"14"" Round HOT-N-READY Pizza",94,200,7,3.5,0,25,340


In [25]:
Pizza_Large14.dtypes
#Pizza = pd.DataFrame.from_dict(Pizza_Large14)

Restaurants       object
Type              object
Serving_Size     float64
Calories         float64
Total_Fat        float64
Saturated_Fat    float64
Trans_Fat        float64
Carbs            float64
Sodium           float64
dtype: object

In [26]:
#CarbsCal = Pizza_Large14['Carbs'] * 4
#FatCal = Pizza_Large14['Total_Fat'] * 9

#CarbsRatio = Pizza_Large14['Calories'] / CarbsCal 
#FatsRatio = Pizza_Large14['Calories'] / FatCal


In [27]:
#CREATE DB
con1 = lite.connect("Final_Project.db")

cur = con1.cursor()


#DROP TABLE IF EXISTS:
cur.execute("DROP TABLE IF EXISTS Fries")
cur.execute("DROP TABLE IF EXISTS Hamburgers")
cur.execute("DROP TABLE IF EXISTS Sandwiches_Hamburger")
cur.execute("DROP TABLE IF EXISTS Chicken_Pieces")
cur.execute("DROP TABLE IF EXISTS Chicken_Sandwiches")
cur.execute("DROP TABLE IF EXISTS Onion_Rings")
cur.execute("DROP TABLE IF EXISTS Bkfst_Sandwiches")
cur.execute("DROP TABLE IF EXISTS Mozzarella_Sticks")
cur.execute("DROP TABLE IF EXISTS BreadSticks_CheesyBread")
cur.execute("DROP TABLE IF EXISTS Pizza_Large14")


#Insert Into Database ALL OF DATAFRAMES

sql.write_frame(Fries, name='Fries', con=con1)
sql.write_frame(Hamburgers, name='Hamburgers', con=con1)
sql.write_frame(Sandwiches_Hamburger, name='Sandwiches_Hamburger', con=con1)
sql.write_frame(Chicken_Pieces, name='Chicken_Pieces', con=con1)
sql.write_frame(Chicken_Sandwiches, name='Chicken_Sandwiches', con=con1)
sql.write_frame(Onion_Rings, name='Onion_Rings', con=con1)
sql.write_frame(Bkfst_Sandwiches, name='Bkfst_Sandwiches', con=con1)
sql.write_frame(Mozzarella_Sticks, name='Mozzarella_Sticks', con=con1)
sql.write_frame(BreadSticks_CheesyBread, name='BreadSticks_CheesyBread', con=con1)
sql.write_frame(Pizza_Large14, name='Pizza_Large14', con=con1)

#Close Connection
con1.close()

/Users/LightWeight/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:22: FutureWarning: write_frame is deprecated, use to_sql
/Users/LightWeight/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:23: FutureWarning: write_frame is deprecated, use to_sql
/Users/LightWeight/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:24: FutureWarning: write_frame is deprecated, use to_sql
/Users/LightWeight/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:25: FutureWarning: write_frame is deprecated, use to_sql
/Users/LightWeight/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:26: FutureWarning: write_frame is deprecated, use to_sql
/Users/LightWeight/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:27: FutureWarning: write_frame is deprecated, use to_sql
/Users/LightWeight/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:28: FutureWarning: write_frame is deprecated, use to_sql
/Users/LightWeight/anaconda/lib/python3.4/site-packages

In [28]:
# TESTING DATABASE:
#CONNECTION:
con2 = lite.connect('Final_Project.db')
cur2 = con2.cursor()

#SQL COMMAND SELECTING TABLE
cur2.execute('SELECT * FROM Fries')

#FETCH ALL ROWS FROM TABLE
rows = cur2.fetchall()

#DISPLAY OF ROWS:
for row in rows:
    print (row, '    :Rows FROM|SQL TABLE')

#CLOSE SQL CONNECTION
con2.close()

("McDonald's", 'Regular', 170.0, 570.0, 30.0, 6.0, 8.0, 70.0, 330.0, 49.122807017543856, 47.368421052631575, 3.508771929824561)     :Rows FROM|SQL TABLE
('Burger King', 'Regular', 160.0, 500.0, 28.0, 6.0, 6.0, 57.0, 820.0, 45.6, 50.4, 4.0)     :Rows FROM|SQL TABLE
('Burger King', 'Salt Not Added', 160.0, 500.0, 28.0, 6.0, 6.0, 57.0, 530.0, 45.6, 50.4, 4.0)     :Rows FROM|SQL TABLE
("Wendy's", 'Regular', 0.0, 540.0, 26.0, 4.0, 1.0, 69.0, 550.0, 51.11111111111111, 43.333333333333336, 5.555555555555555)     :Rows FROM|SQL TABLE
("Arby's", 'Homestyle', 213.0, 566.0, 37.0, 7.0, 1.0, 82.0, 1029.0, 57.95053003533569, 58.8339222614841, -16.784452296819786)     :Rows FROM|SQL TABLE
("Arby's", 'Curly', 198.0, 631.0, 37.0, 7.0, 1.0, 73.0, 1476.0, 46.275752773375594, 52.77337559429477, 0.9508716323296356)     :Rows FROM|SQL TABLE
("Hardee's", 'Regular', 193.0, 610.0, 28.0, 6.0, 0.0, 78.0, 370.0, 51.147540983606554, 41.31147540983607, 7.540983606557377)     :Rows FROM|SQL TABLE
("Hardee's", 'Crispy